<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 15px; height: 80px">

# Project 6:  Web Scraping
### Finding Underpriced RVs on Craigslist

![](https://snag.gy/WrdUMx.jpg)

In this project we will be practicing our web scraping skills.  You can use Scrapy or Python requests in order to complete this project.  It may be helpful to write some prototype code in this notebook to test your assumptions, then move it into a Python file that can be run from the command line.

> In order to run code from the command line, instead of the notebook, you just need to save your code to a file (with a .py extension), and run it using the Python interpreter:<br><br>
> `python my_file.py`

You will be building a process to scrape a single category of search results on Craigslist, that can easily be applied to other categories by changing the search terms.  The main goal is to be able to target and scrape a single page given a set of parameters.

**If you use Scrapy, provide your code in a folder.**

## Import your libraries for scrapy / requests / pandas / numpy / etc
Setup whichever libraries you need. Review past material for reference.

In [1]:
# PREPARE REQUIRED LIBRARIES

import pandas as pd
import numpy as np
from scrapy.selector import Selector
from scrapy.http import HtmlResponse
import requests

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 1.  Scrape for the largest US cities (non-exhaustive list)
Search, research, and scrape Wikipedia for a list of the largest US cities.  There are a few sources but find one that is in a nice table.  We don't want all cities, just signifficant cities.  Examine your source.  Look for what can be differentiable.

- Use requests
- Build XPath query(ies)
- Extract to a list
- Clean your list

In [2]:
# SCRAPE WIKIPEDIA FOR LARGEST US CITIES (NON-EXHAUSTIVE LIST)
url = "https://simple.wikipedia.org/wiki/List_of_United_States_cities_by_population"
response = requests.get(url)

HTML = response.text

In [3]:
cities = Selector(text=HTML).xpath('//tr/td[2]/a/text()').extract()

In [4]:
cities

[u'New York',
 u'Los Angeles',
 u'Chicago',
 u'Houston',
 u'Philadelphia',
 u'Phoenix',
 u'San Antonio',
 u'San Diego',
 u'Dallas',
 u'San Jose',
 u'Austin',
 u'Jacksonville',
 u'San Francisco',
 u'Indianapolis',
 u'Columbus',
 u'Fort Worth',
 u'Charlotte',
 u'Detroit',
 u'El Paso',
 u'Seattle',
 u'Denver',
 u'Washington',
 u'Memphis',
 u'Boston',
 u'Nashville',
 u'Baltimore',
 u'Oklahoma City',
 u'Portland',
 u'Las Vegas',
 u'Louisville',
 u'Milwaukee',
 u'Albuquerque',
 u'Tucson',
 u'Fresno',
 u'Sacramento',
 u'Long Beach',
 u'Kansas City',
 u'Mesa',
 u'Atlanta',
 u'Virginia Beach',
 u'Omaha',
 u'Colorado Springs',
 u'Raleigh',
 u'Miami',
 u'Oakland',
 u'Minneapolis',
 u'Tulsa',
 u'Cleveland',
 u'Wichita',
 u'New Orleans',
 u'Arlington',
 u'Bakersfield',
 u'Tampa',
 u'Aurora',
 u'Honolulu',
 u'Anaheim',
 u'Santa Ana',
 u'Corpus Christi',
 u'Riverside',
 u'St. Louis',
 u'Lexington',
 u'Pittsburgh',
 u'Stockton',
 u'Anchorage',
 u'Cincinnati',
 u'Saint Paul',
 u'Greensboro',
 u'Toledo'

<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 1.2 Only retain cities with properly formed ASCII

Optionally, filter out any cities with impropper ASCII characters.  A smaller list will be easier to look at.  However you may not need to filter these if you spend more time scraping a more concise city list.  This list should help you narrow down the list of regional Craigslist sites.

In [5]:
cities = [city.encode('ascii',errors='ignore') for city in cities]

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 2.  Write a function to capture current pricing information via Craigslist in one city.
Choose a city from your scraped data, then go to the cooresponding city section on Craigslist, searching for "rv" in the auto section.  Write a method that pulls out the prices.

In [65]:
def price_grabber(city):
    url = 'https://%s.craigslist.org/search/rva' % (city)
    response = requests.get(url)
    HTML = response.text
    prices = Selector(text=HTML).xpath('//span[@class="l2"]/span[@class="price"]/text()').extract()
    prices = [int(x.replace('$','')) for x in prices]
    return prices
    

price_grabber("sfbay")[0:5]

[7950, 6000, 1, 6896, 80, 1, 12500]

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 2.1 Create a mapping of cities to cooresponding regional Craigslist URLs

Major US cities on Craigslist typically have their own cooresponding section (ie: SFBay Area, NYC, Boston, Miami, Seattle, etc).  Later, you will use these to query search results for various metropolitian regions listed on Craigslist.  Between the major metropolitan Craigslist sites, the only thing that will differ is the URL's that correspond to them.

The point of the "mapping":  Create a data structure that allows you to iterate with both the name of the city from Wikipedia, with the cooresponding variable that that will allow you to construct each craigslist URL for each region.

> For San Francsico (the Bay Area metropolitan area), the url for the RV search result is:
> http://sfbay.craigslist.org/search/sss?query=rv
>
> The convention is http://[region].craigslist.org/search/sss?query=rf
> Replacing [region] with the cooresponding city name will allow you to quickly iterate through each regional Craigslist site, and scrape the prices from the search results.  Keep this in mind while you build this "mapping".


In [69]:
cities_list = [city.replace(' ', '').lower() for city in cities]

cities_list = cities_list[:9] 

cities_list 

['newyork',
 'losangeles',
 'chicago',
 'houston',
 'philadelphia',
 'phoenix',
 'sanantonio',
 'sandiego',
 'dallas']

In [70]:
def mapping(cities_list):
    city_urls = {}
    
    for city in cities_list:
        url = 'https://%s.craigslist.org/search/rva' %(city)
        city_urls[city] = url
    return city_urls

city_map = mapping(cities_list)

In [71]:
city_map

{'chicago': 'https://chicago.craigslist.org/search/rva',
 'dallas': 'https://dallas.craigslist.org/search/rva',
 'houston': 'https://houston.craigslist.org/search/rva',
 'losangeles': 'https://losangeles.craigslist.org/search/rva',
 'newyork': 'https://newyork.craigslist.org/search/rva',
 'philadelphia': 'https://philadelphia.craigslist.org/search/rva',
 'phoenix': 'https://phoenix.craigslist.org/search/rva',
 'sanantonio': 'https://sanantonio.craigslist.org/search/rva',
 'sandiego': 'https://sandiego.craigslist.org/search/rva'}

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 3. Define a function to caculate mean and median price per city.

Now that you've created a list of cities you want to scrape, adapt your solution for grabbing data in one region site, to grab data for all regional sites that you collected, then calculate the mean and median price of RV results from each city.

> Look at the URLs from a few different regions (ie: portland, phoenix, sfbay), and find what they have in common.  Determine the area in the URL string that needs to change the least, and figure out how to replace only that portion of the URL in order to iterate through each city.

In [72]:
def price_mean_median(city):
    price = price_grabber(city)
    return np.mean(price), np.median(price)

In [82]:
#I severely limited my cities list because fortworth & san jose were giving me trouble... use a try & except but how?
#also how can I get the results to 2 decimals points? tried a few things ie  %.2f but didn't work??
for city in city_map:
    print city, 'mean/median price', price_mean_median(city)

houston mean/median price (19189.54054054054, 12995.0)
losangeles mean/median price (24409.976190476191, 8749.5)
sandiego mean/median price (18950.494252873563, 8999.0)
phoenix mean/median price (28786.599999999999, 9725.0)
chicago mean/median price (12518.46875, 7100.0)
newyork mean/median price (22117.010989010989, 10700.0)
dallas mean/median price (29655.614285714284, 20972.5)
philadelphia mean/median price (33407.558139534885, 14216.0)
sanantonio mean/median price (37412.016949152545, 19340.0)


<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 4. Run your scraping process, and save your results to a CSV file.

In [89]:
#not entirely sure how to get this into a dataframe... 
#after getting a dataframe I'd use df.to_csv('prices.csv') to export a csv file
def results(city):
    price = price_grabber(city)
    return price

In [87]:
for city in city_map:
    print city, results(city)

houston [14000, 45000, 4800, 7750, 5900, 12800, 14, 12500, 13900, 79900, 2500, 25000, 27500, 85, 58700, 16900, 16500, 62150, 800, 129, 12995, 8995, 4000, 7900, 12250, 11700, 35000, 1900, 35000, 35000, 20500, 35000, 35000, 8000, 17950, 14995, 7000]
losangeles [8500, 109891, 44891, 10100, 10100, 975, 63891, 22000, 75891, 10000, 27500, 9950, 9750, 17950, 16500, 5000, 18500, 8999, 1, 79000, 20000, 6200, 120000, 34950, 22500, 1950, 5000, 5000, 12200, 550, 7000, 8500, 12950, 5000, 120000, 89, 79, 5000, 15500, 3200, 5000, 45000, 17500, 3800, 1, 25, 15000, 7995, 19637, 6000, 120000, 120000, 120000, 64495, 35, 4500, 795, 2000, 2189, 600, 600, 395, 200, 24681, 1200, 52000, 79, 54366, 4800, 4750, 9500, 6000, 18, 109000, 250, 6500, 100, 2045, 11350, 19995, 7995, 14995, 119995, 89995]
sandiego [150, 150, 12500, 29500, 150, 10100, 10100, 23995, 8900, 100, 9500, 23000, 7000, 250, 8500, 10500, 8999, 1200, 22500, 11999, 44995, 16500, 280, 1950, 650, 29990, 24500, 18500, 78000, 20000, 13000, 6500, 10950

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 5. Do an analysis of the RV market.

Go head we'll wait.  Anything notable about the data?

In [88]:
#obviously some graphs would be nice... but modeling? what model? a simple linear regression? could I do something else

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 5.1 Does it makes sense to buy RVs in one region and sell them in another?

Assuming the cost of shipping or driving from one regional market to another.

<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 5.2 Can you pull out the "make" from the markup and include that in your analyis?
How reliable is this data and does it make sense?

<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 5.3 Are there any other variables you could pull out of the markup to help describe your dataset?

<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 6. Move your project into scrapy (if you haven't used Scrapy yet)

>Start a project by using the command `scrapy startproject [projectname]`
> - Update your settings.py (review our past example)
> - Update your items.py
> - Create a spiders file in your `[project_name]/[project_name]/spiders` directory

You can update your spider class with the complete list of craigslist "start urls" to effectively scrape all of the regions.  Start with one to test.

Updating your parse method with the method you chose should require minimal changes.  It will require you to update your parse method to use the response parameter, and an item model (defined in items.py).

<img src="http://imgur.com/GCAf1UX.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 7.  Chose another area of Craigslist to scrape.

**Choose an area having more than a single page of results, then scrape multiple regions, multiple pages of search results and or details pages.**

This is the true exercise of being able to understand how to succesffuly plan, develop, and employ a broader scraping strategy.  Even though this seems like a challenging task, a few tweeks of your current code can make this very managable if you've pieced together all the touch points.  If you are still confused as to some of the milestones within this process, this is an excellent opportunity to round out your understanding, or help you build a list of questions to fill in your gaps.

_Use Scrapy!  Provide your code in this project directory when you submit this project._